<a href="https://colab.research.google.com/github/kangwonlee/momisp/blob/main/Ch05_Stress.in.Beams/ex05.006.numpy_sympy.ShearStress.rect.section_simple_w.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.



`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)



In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비



## 예제 05.006



### 문제에서 주어진 변수<br>Given Parameters



#### 각 구간 길이<br>Length of each section



In [ ]:
L_AB_m = sy.symbols('L_{AB}[m]', positive=True)

s_d = {L_AB_m: 6}



#### 단면<br>Section



In [ ]:
b_m, h_wood_m, h_m = sy.symbols('b[m], h_{wood}[m], h[m]', positive=True)
# Nick Fortescue, Jacob Marble, '''Python “extend” for a dictionary''', Stackoverflow, 2013 March 23, https://stackoverflow.com/questions/577234/python-extend-for-a-dictionary
s_d.update({b_m: 160e-3,
            h_wood_m: 80e-3,
            h_m: 80e-3 * 3})



#### 반력<br>Reaction Force



In [ ]:
R_A_N, R_B_N = sy.symbols('R_{A}[N], R_{B}[N]', real=True)

s_d.update({R_A_N: 24e3,
            R_B_N: 24e3})



#### 하중<br>Load



In [ ]:
w_AB_N_m = sy.Symbol('w_{AB}[N/m]')

s_d[w_AB_N_m] = -8e3



### 각 지점의 x 좌표



In [ ]:
x_m = sy.Symbol('x[m]')



A 점에서 x=0 으로 한다.<br>Let x=0 at A point



In [ ]:
x_A_m = 0



In [ ]:
x_B_m = x_A_m + L_AB_m.subs(s_d)



### 전단력선도<br>Shear Force Diagram



In [ ]:
V_AB_N = R_A_N + w_AB_N_m * x_m
V_AB_N



In [ ]:
V_AB_N.subs(s_d)



선도로 표시해 보자.



In [ ]:
splot.plot((V_AB_N.subs(s_d), (x_m, 0, x_B_m)))



### (1) 접착부위의 전단응력



* 중립면의 전단응력 eq. 5.8 (p. 199)



$$
\tau(Pa)=\frac{V(N)Q(m^3)}{b(m)I(m^4)}
$$



* 전단력<br>Shear Force



In [ ]:
V_N = sy.Symbol('V[N]', real=True)



* 접착면 위의 면적



In [ ]:
A_wood_m2 = b_m * h_wood_m



* 접착면 위 면적의 도심과 중립면 사이의 거리



In [ ]:
y_prime_wood_bar_m = h_wood_m/2 + h_wood_m/2



* 접착면 위 면적의 모멘트 $Q$



In [ ]:
Q_wood_m3 = A_wood_m2 * y_prime_wood_bar_m
Q_wood_m3



In [ ]:
Q_wood_m3.subs(s_d)



* 전체 단면의 I



In [ ]:
I_m4 = (b_m * h_m ** 3) / 12



In [ ]:
I_m4.subs(s_d)



* 접착면의 전단응력



In [ ]:
tau_wood_Pa = V_N * Q_wood_m3 / (b_m * I_m4)
tau_wood_Pa



In [ ]:
tau_wood_Pa.subs(V_N, R_A_N).subs(s_d)



### (2) 최대 전단응력



* $y = 0$ 인 중립면에서 발생할 것이다.



* 중립면 위의 면적<br>Area above the neutral plane



In [ ]:
A_m2 = b_m * h_m / 2



* 중립면 위 면적의 도심과 중립면 사이의 거리



In [ ]:
y_prime_bar_m = h_m/4



* 중립면 위 면적의 모멘트 $Q$



In [ ]:
Q_max_m3 = A_m2 * y_prime_bar_m
Q_max_m3



In [ ]:
Q_max_m3.subs(s_d)



* 중립면의 전단응력 eq. 5.8 (p. 199)



$$
\tau_{max}(Pa)=\frac{V(N)Q_{max}(m^3)}{b(m)I(m^4)}
$$



In [ ]:
tau_max_Pa = V_N * Q_max_m3 / (b_m * I_m4)
tau_max_Pa



위 결과는 eq. 5.9 (p. 201) 와 일치하는가?



In [ ]:
tau_max_Pa.subs(V_N, R_A_N).subs(s_d)



### 전단응력의 분포<br>Distribution of shear stress



`x` 좌표 배열 준비<br>`x` coordinate array



In [ ]:
dx_m = 10e-3
dx_half_m = dx_m * 0.5
x_m_array = np.arange(0, s_d[L_AB_m] + dx_half_m, dx_m)



`y` 좌표 배열 준비<br>`y` coordinate array



In [ ]:
dy_m = 1e-3
dy_half_m = dy_m * 0.5
y_m_array = np.arange(s_d[h_m] * (-0.5), s_d[h_m] * (0.5) + dy_half_m, dy_m)



`x` `y` 좌표 격자 준비<br>`x` `y` coordinate grids



In [ ]:
x_m_grid, y_m_grid = np.meshgrid(x_m_array, y_m_array)



`x` `y` 격자 크기<br>Shape of `x` `y` grids



In [ ]:
x_m_grid.shape, y_m_grid.shape,



`x` `y` 격자 크기가 같은지 확인함<br>Check if `x` and `y` grids have the same shapes



In [ ]:
assert (x_m_grid.shape == y_m_grid.shape)



전단력 배열 또는 격자를 계산하기 위한 함수를 생성하도록 함.<br>Make `sympy` generate a function to calulate shear force array or grid.



In [ ]:
f_V_AB_N = sy.lambdify(x_m, V_AB_N.subs(s_d))



전단력 격자를 계산함.<br>Calculate shear force grid.



In [ ]:
V_AB_N_grid = f_V_AB_N(x_m_grid)



$Q$ 값을 계산하기 위해 중립면에서 `y` 만큼 떨어진 넓이를 계산.<br>Calculate area whose distance from the neutral axis is `y` to obtain $Q$. 



In [ ]:
A_m2_grid = s_d[b_m] * (s_d[h_m] * 0.5 - abs(y_m_grid))



`y` 와 그 위 면적을 그림으로 표시.<br>Plot `y` vs area above it.



In [ ]:
plt.plot(A_m2_grid[:,0], y_m_grid[:,0])
plt.xlabel('$A(m^2)$')
plt.ylabel('y(m)')
plt.xlim(xmin=0)
plt.grid(True)
plt.show()



`y`와 `y` 위 면적의 도심의 위치를 그림으로 표시.<br>Plot `y` vs centroid of area above `y`.



In [ ]:
y_bar_m_grid = s_d[h_m] * 0.25 + abs(y_m_grid) * 0.5



`y` 격자와 `y_bar` 격자의 크기를 비교.<br>Compare the shapes of the `y` and `y_bar` grids.



In [ ]:
y_m_grid.shape, y_bar_m_grid.shape



In [ ]:
assert (y_m_grid.shape == y_bar_m_grid.shape)



In [ ]:
plt.plot(y_bar_m_grid[:,0], y_m_grid[:,0])
plt.xlabel('$\overline{y\'}(m)$')
plt.ylabel('y(m)')
plt.xlim(xmin=0)
plt.grid(True)
plt.show()



$Q$ 값을 계산<br>Calculate $Q$



In [ ]:
Q_m3_grid = (A_m2_grid * y_bar_m_grid)



In [ ]:
plt.plot(Q_m3_grid[:,0], y_m_grid[:,0])
plt.xlabel('$Q(m^3)$')
plt.ylabel('y(m)')
plt.xlim(xmin=0)
plt.grid(True)
plt.show()



전단응력을 계산함<br>Calculate the shear stress



In [ ]:
tau_Pa_grid = (V_AB_N_grid) * Q_m3_grid / float((s_d[b_m] * I_m4.subs(s_d)))



전단응력의 분포를 표시함<br>Plot the distribution of the shear stress



In [ ]:
plt.pcolormesh(x_m_grid, y_m_grid, tau_Pa_grid * 1e-6, shading="auto", cmap='jet')
plt.xlabel('x(m)')
plt.ylabel('y(m)')
cbar = plt.colorbar()
cbar.set_label('$\\tau(MPa)$')
plt.show()

